# Fiddler OpenTelemetry Integration - Advanced Guide

## Goal

This comprehensive guide demonstrates how to integrate **custom AI agents** with **Fiddler** using **OpenTelemetry** for full observability and monitoring. You will learn advanced patterns for instrumentation, attribute mapping, and production-ready configurations. 🤖

## About OpenTelemetry Integration

OpenTelemetry provides a vendor-neutral way to collect telemetry data from your applications. This notebook shows you how to:

- Set up OpenTelemetry tracing with Fiddler's OTLP endpoint
- Map agent attributes to Fiddler's semantic conventions
- Instrument LLM calls, tool execution, and agent chains
- Track conversations and sessions with custom attributes
- Implement production-ready configurations
- Debug and troubleshoot trace collection

## About Fiddler

Fiddler is the all-in-one AI Observability and Security platform for responsible AI. Monitoring and analytics capabilities provide a common language, centralized controls, and actionable insights to operationalize production ML models, GenAI, AI agents, and LLM applications with trust.

## When to Use OpenTelemetry Integration

Use this approach when:

- **Multi-framework environments**: You use multiple agentic frameworks and need unified observability
- **Custom frameworks**: Your agent framework doesn't have a dedicated Fiddler SDK
- **Advanced control**: You need fine-grained control over instrumentation

**When to Use SDKs Instead:**

- **LangGraph/LangChain**: Use [Fiddler LangGraph SDK](https://docs.fiddler.ai/tutorials/llm-monitoring/langgraph-sdk-quick-start) for auto-instrumentation
- **Strands Agents**: Use [Fiddler Strands SDK](https://docs.fiddler.ai/tutorials/llm-monitoring/strands-agent-quick-start) for native integration

---

## Notebook Summary

In this notebook, we will:

1. Install OpenTelemetry packages
2. Set up environment variables for Fiddler OTLP endpoint
3. Configure OpenTelemetry with TracerProvider and OTLP exporter
4. Build a comprehensive travel agent with hotel and flight booking tools
5. Implement custom user-defined attributes
6. Track conversations across multi-turn interactions
7. Configure production settings (sampling, batching, compression)
8. Verify traces in Fiddler dashboard

**Time to complete:** ~25-30 minutes

---

## Prerequisites

- Fiddler account with Application UUID and Access Token
- OpenAI API key (or other LLM provider)
- Python 3.10+
- OpenTelemetry packages

## 1. Install Required Packages

First, install OpenTelemetry and LLM provider packages.

In [ ]:
# Install OpenTelemetry packages
%pip install -q opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp-proto-http openai

## 2. Configure Environment Variables

Set up your Fiddler credentials and LLM API key.

**Where to find these values:**

- **FIDDLER_URL**: Your Fiddler instance URL (e.g., `https://acme.fiddler.ai`)
- **APPLICATION_UUID**: From your Fiddler GenAI Apps → Application Details
- **ACCESS_TOKEN**: From Fiddler Settings → Credentials
- **OPENAI_API_KEY**: Your OpenAI API key

In [ ]:
import os

# ⚠️ Replace these with your actual credentials
FIDDLER_URL = 'https://your-instance.fiddler.ai'
APPLICATION_UUID = 'your-application-uuid-here'  # Must be valid UUID4
ACCESS_TOKEN = 'your-access-token-here'
OPENAI_API_KEY = 'your-openai-api-key-here'

# Set environment variables for OpenTelemetry
os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'] = FIDDLER_URL
os.environ['OTEL_EXPORTER_OTLP_HEADERS'] = (
    f'authorization=Bearer {ACCESS_TOKEN},'
    f'fiddler-application-id={APPLICATION_UUID}'
)
os.environ['OTEL_RESOURCE_ATTRIBUTES'] = f'application.id={APPLICATION_UUID}'
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print('✅ Environment variables configured')
print(f'   Fiddler URL: {FIDDLER_URL}')
print(f'   Application ID: {APPLICATION_UUID}')

## 3. Part 1: Basic OpenTelemetry Configuration

Set up OpenTelemetry with:

- **TracerProvider**: Manages trace generation
- **OTLPSpanExporter**: Exports spans to Fiddler
- **BatchSpanProcessor**: Batches spans for efficient transmission
- **Console Exporter**: For local debugging (optional)

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

# Initialize tracer provider
trace.set_tracer_provider(TracerProvider())
tracer = trace.get_tracer(__name__)

# Configure OTLP exporter for Fiddler
otlp_endpoint = os.getenv('OTEL_EXPORTER_OTLP_ENDPOINT') + '/v1/traces'
otlp_exporter = OTLPSpanExporter(endpoint=otlp_endpoint)
otlp_processor = BatchSpanProcessor(otlp_exporter)
trace.get_tracer_provider().add_span_processor(otlp_processor)

# Optional: Add console exporter for local debugging
console_exporter = ConsoleSpanExporter()
console_processor = BatchSpanProcessor(console_exporter)
trace.get_tracer_provider().add_span_processor(console_processor)

print('✅ OpenTelemetry configured successfully')
print(f'   OTLP Endpoint: {otlp_endpoint}')
print('   Console Exporter: Enabled (for debugging)')

## 4. Part 2: Comprehensive Travel Agent Example

This section implements a production-ready travel agent with:

- Hotel booking tool
- Flight booking tool
- LLM orchestration
- Proper span hierarchy (chain → LLM → tools)
- Complete Fiddler attribute mapping

### Define Agent Constants and System Prompt

In [ ]:
AGENT_NAME = "travel_agent"
AGENT_ID = "travel_agent_v1"

TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a professional travel agent capable of booking hotels and flights.
You have access to the following tools: book_hotel_tool, book_flight_tool.

When a user asks to book travel, you should:
1. Search for available hotels if they need accommodation
2. Book the hotel if they approve
3. Book flights if they need transportation
4. Provide confirmation details for all bookings

Be helpful, professional, and ensure you gather all necessary information
before making any bookings.
"""

print('✅ Agent configuration defined')
print(f'   Agent Name: {AGENT_NAME}')
print(f'   Agent ID: {AGENT_ID}')

### Define Travel Agent Tools

Each tool creates a properly instrumented span with Fiddler-required attributes.

In [ ]:
import json

def _book_hotel(city: str, date: str) -> dict:
    """Simulate hotel booking API."""
    return {
        "status": "confirmed",
        "hotel_name": f"Grand Hotel {city}",
        "city": city,
        "date": date,
        "confirmation": f"HTL-{city[:3].upper()}-{date.replace('-', '')}",
        "price": 250
    }

def book_hotel_tool(city: str, date: str):
    """
    Book a hotel in the specified city.
    Creates an instrumented tool span with Fiddler attributes.
    """
    with tracer.start_as_current_span("book_hotel_tool") as span:
        # Required Fiddler attributes
        span.set_attribute("fiddler.span.type", "tool")
        span.set_attribute("gen_ai.agent.name", AGENT_NAME)
        span.set_attribute("gen_ai.agent.id", AGENT_ID)

        # Tool-specific attributes
        span.set_attribute("gen_ai.tool.name", "book_hotel_tool")
        tool_input = {"city": city, "date": date}
        span.set_attribute("gen_ai.tool.input", json.dumps(tool_input))

        # Execute tool
        result = _book_hotel(city, date)
        span.set_attribute("gen_ai.tool.output", json.dumps(result))

        return result

def _book_flight(source: str, destination: str, date: str) -> dict:
    """Simulate flight booking API."""
    return {
        "status": "confirmed",
        "flight_number": f"FL{source[:2].upper()}{destination[:2].upper()}456",
        "source": source,
        "destination": destination,
        "date": date,
        "departure": "10:00 AM",
        "arrival": "2:30 PM",
        "price": 450
    }

def book_flight_tool(source: str, destination: str, date: str):
    """
    Book a flight between two cities.
    Creates an instrumented tool span with Fiddler attributes.
    """
    with tracer.start_as_current_span("book_flight_tool") as span:
        # Required Fiddler attributes
        span.set_attribute("fiddler.span.type", "tool")
        span.set_attribute("gen_ai.agent.name", AGENT_NAME)
        span.set_attribute("gen_ai.agent.id", AGENT_ID)

        # Tool-specific attributes
        span.set_attribute("gen_ai.tool.name", "book_flight_tool")
        tool_input = {
            "source": source,
            "destination": destination,
            "date": date
        }
        span.set_attribute("gen_ai.tool.input", json.dumps(tool_input))

        # Execute tool
        result = _book_flight(source, destination, date)
        span.set_attribute("gen_ai.tool.output", json.dumps(result))

        return result

print('✅ Travel agent tools defined')
print('   - book_hotel_tool')
print('   - book_flight_tool')

### Implement Main Travel Agent Function

This function orchestrates LLM calls and tool execution with proper instrumentation.

In [ ]:
from openai import OpenAI

client = OpenAI()

def travel_agent(user_request: str):
    """
    Main travel agent function that processes user requests.
    Creates a chain span with nested LLM and tool spans.
    """
    with tracer.start_as_current_span("travel_agent_chain") as root_span:
        # Root chain span
        root_span.set_attribute("fiddler.span.type", "chain")
        root_span.set_attribute("gen_ai.agent.name", AGENT_NAME)
        root_span.set_attribute("gen_ai.agent.id", AGENT_ID)

        # First LLM call: Understand request and plan actions
        with tracer.start_as_current_span("llm_planning") as llm_span:
            # Required Fiddler attributes
            llm_span.set_attribute("fiddler.span.type", "llm")
            llm_span.set_attribute("gen_ai.agent.name", AGENT_NAME)
            llm_span.set_attribute("gen_ai.agent.id", AGENT_ID)

            # LLM-specific attributes
            model = "gpt-4o-mini"
            llm_span.set_attribute("gen_ai.request.model", model)
            llm_span.set_attribute("gen_ai.system", "openai")
            llm_span.set_attribute("gen_ai.llm.input.system", TRAVEL_AGENT_SYSTEM_PROMPT)
            llm_span.set_attribute("gen_ai.llm.input.user", user_request)
            
            # Message history
            llm_span.set_attribute("gen_ai.input.messages", json.dumps([
                {"role": "system", "content": TRAVEL_AGENT_SYSTEM_PROMPT},
                {"role": "user", "content": user_request}
            ]))

            # Set LLM context
            llm_span.set_attribute("gen_ai.llm.context", "Quick start opentelemetry example context")

            # Define tool definitions (reusable)
            tool_definitions = [
                {
                    "type": "function",
                    "function": {
                        "name": "book_hotel_tool",
                        "description": "Book a hotel in a city for a specific date",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "city": {"type": "string"},
                                "date": {"type": "string"}
                            },
                            "required": ["city", "date"]
                        }
                    }
                },
                {
                    "type": "function",
                    "function": {
                        "name": "book_flight_tool",
                        "description": "Book a flight between two cities",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "source": {"type": "string"},
                                "destination": {"type": "string"},
                                "date": {"type": "string"}
                            },
                            "required": ["source", "destination", "date"]
                        }
                    }
                }
            ]

            # NEW (v1.1.0+): Capture tool definitions on span
            llm_span.set_attribute("gen_ai.tool.definitions", json.dumps(tool_definitions))

            # Call OpenAI with tool definitions
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": TRAVEL_AGENT_SYSTEM_PROMPT},
                    {"role": "user", "content": user_request}
                ],
                tools=tool_definitions
            )

            # Set token usage
            llm_span.set_attribute("gen_ai.usage.input_tokens", response.usage.prompt_tokens)
            llm_span.set_attribute("gen_ai.usage.output_tokens", response.usage.completion_tokens)
            llm_span.set_attribute("gen_ai.usage.total_tokens", response.usage.total_tokens)

            # Process tool calls
            tool_results = []
            message = response.choices[0].message

            if message.tool_calls:
                # Build tool_call parts for complex OpenTelemetry format
                tool_call_parts = []
                for tool_call in message.tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = json.loads(tool_call.function.arguments)

                    # Execute the tool
                    if tool_name == "book_hotel_tool":
                        result = book_hotel_tool(**tool_args)
                        tool_results.append({"tool": "hotel", "result": result})
                    elif tool_name == "book_flight_tool":
                        result = book_flight_tool(**tool_args)
                        tool_results.append({"tool": "flight", "result": result})
                    
                    # Add tool call part (complex format with type, name, id, arguments)
                    tool_call_parts.append({
                        "type": "tool_call",
                        "name": tool_name,
                        "id": tool_call.id,
                        "arguments": tool_args
                    })

                llm_span.set_attribute(
                    "gen_ai.llm.output",
                    f"Executed {len(tool_results)} tool(s): {[t['tool'] for t in tool_results]}"
                )
                
                # Output message history (complex format with tool calls)
                llm_span.set_attribute("gen_ai.output.messages", json.dumps([{
                    "role": "assistant",
                    "parts": tool_call_parts,
                    "finish_reason": response.choices[0].finish_reason
                }]))
            else:
                llm_span.set_attribute("gen_ai.llm.output", message.content or "No response")
                
                # Output message history (simple format)
                llm_span.set_attribute("gen_ai.output.messages", json.dumps([
                    {"role": "assistant", "content": message.content or ""}
                ]))

        # Second LLM call: Generate user-friendly response
        if tool_results:
            with tracer.start_as_current_span("llm_response_generation") as response_span:
                response_span.set_attribute("fiddler.span.type", "llm")
                response_span.set_attribute("gen_ai.agent.name", AGENT_NAME)
                response_span.set_attribute("gen_ai.agent.id", AGENT_ID)
                response_span.set_attribute("gen_ai.request.model", model)
                response_span.set_attribute("gen_ai.system", "openai")

                summary_prompt = f"Summarize these booking results for the user: {json.dumps(tool_results)}"
                response_span.set_attribute("gen_ai.llm.input.user", summary_prompt)
                
                # Message history
                response_span.set_attribute("gen_ai.input.messages", json.dumps([
                    {"role": "system", "content": "Provide a friendly booking confirmation summary."},
                    {"role": "user", "content": summary_prompt}
                ]))

                final_response = client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": "Provide a friendly booking confirmation summary."},
                        {"role": "user", "content": summary_prompt}
                    ]
                )

                response_span.set_attribute("gen_ai.usage.input_tokens", final_response.usage.prompt_tokens)
                response_span.set_attribute("gen_ai.usage.output_tokens", final_response.usage.completion_tokens)
                response_span.set_attribute("gen_ai.usage.total_tokens", final_response.usage.total_tokens)

                final_text = final_response.choices[0].message.content
                response_span.set_attribute("gen_ai.llm.output", final_text)
                
                # Output message history (simple format)
                response_span.set_attribute("gen_ai.output.messages", json.dumps([
                    {"role": "assistant", "content": final_text}
                ]))

                return {
                    "status": "success",
                    "bookings": tool_results,
                    "summary": final_text
                }

        return {
            "status": "no_action",
            "message": message.content if message.content else "No tools executed"
        }

print('✅ Travel agent implementation complete')
print('   Supports: hotel booking, flight booking, LLM orchestration')

### Test the Travel Agent

Run a sample query to verify instrumentation works.

In [ ]:
# Test query
result = travel_agent("I need to book a hotel in Paris for 2024-12-15 and a flight from London to Paris for the same day")

print('\n' + '=' * 60)
print('TRAVEL AGENT RESPONSE')
print('=' * 60)
print(f'Status: {result["status"]}')
if result["status"] == "success":
    print(f'\nBookings Made: {len(result["bookings"])}')
    for booking in result["bookings"]:
        print(f"  - {booking['tool']}: {json.dumps(booking['result'], indent=4)}")
    print(f'\nSummary: {result["summary"]}')
print('=' * 60)

### Understanding Tool Definitions Capture (v1.1.0+)

Starting with **Fiddler LangGraph SDK v1.1.0**, you can capture tool definitions that are available to the LLM for tool calling. This enables powerful analysis and evaluation capabilities:

#### Why Capture Tool Definitions?

- **Validate Tool Usage:** Verify that the LLM selected the correct tool for the task
- **Parameter Validation:** Check that all required parameters were provided correctly
- **Tool Coverage Analysis:** Understand which tools are being used and which are ignored
- **Debugging:** Quickly identify tool-related issues in production
- **Evaluation:** Enable automated evaluators to validate tool calls against their schemas

#### How It Works

1. **Define tools** in OpenAI format (type, function name, description, parameters)
2. **Capture on span** by setting `gen_ai.tool.definitions` attribute with JSON-serialized tools
3. **View in Fiddler** - Tool definitions appear in the trace viewer and are available for evaluation

#### Format

Tool definitions are stored in OpenAI's native nested format:

```json
[
  {
    "type": "function",
    "function": {
      "name": "tool_name",
      "description": "What the tool does",
      "parameters": {
        "type": "object",
        "properties": {
          "param1": {"type": "string", "description": "..."},
          "param2": {"type": "number", "description": "..."}
        },
        "required": ["param1"]
      }
    }
  }
]
```

#### Best Practices

- Always capture tool definitions when using tool calling
- Include detailed descriptions for better LLM tool selection
- Specify parameter types and required fields accurately
- Keep tool definitions in sync with actual tool implementations
- Remember to JSON-serialize before setting as span attribute

Let's see a standalone example:

In [ ]:
# Standalone example: Tool definitions capture for any LLM call

def make_llm_call_with_tools(prompt: str, tools: list):
    """
    Example function showing how to capture tool definitions
    for any LLM call with tool calling capability.
    """
    with tracer.start_as_current_span("llm_with_tools") as span:
        # Set basic attributes
        span.set_attribute("fiddler.span.type", "llm")
        span.set_attribute("gen_ai.agent.name", AGENT_NAME)
        span.set_attribute("gen_ai.agent.id", AGENT_ID)
        span.set_attribute("gen_ai.request.model", "gpt-4o-mini")
        span.set_attribute("gen_ai.system", "openai")
        span.set_attribute("gen_ai.llm.input.user", prompt)
        
        # Message history
        span.set_attribute("gen_ai.input.messages", json.dumps([
            {"role": "user", "content": prompt}
        ]))
        
        # Capture tool definitions (v1.1.0+)
        span.set_attribute("gen_ai.tool.definitions", json.dumps(tools))
        
        # Make LLM call
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            tools=tools
        )
        
        # Set output
        if response.choices[0].message.tool_calls:
            # Build tool_call parts
            tool_call_parts = []
            for tc in response.choices[0].message.tool_calls:
                tool_call_parts.append({
                    "type": "tool_call",
                    "name": tc.function.name,
                    "id": tc.id,
                    "arguments": json.loads(tc.function.arguments)
                })
            
            span.set_attribute("gen_ai.llm.output", f"Called tools: {[tc['name'] for tc in tool_call_parts]}")
            
            # Output message history (complex format with tool calls)
            span.set_attribute("gen_ai.output.messages", json.dumps([{
                "role": "assistant",
                "parts": tool_call_parts,
                "finish_reason": response.choices[0].finish_reason
            }]))
        else:
            span.set_attribute("gen_ai.llm.output", response.choices[0].message.content or "")
            
            # Output message history (simple format)
            span.set_attribute("gen_ai.output.messages", json.dumps([
                {"role": "assistant", "content": response.choices[0].message.content or ""}
            ]))
        
        span.set_attribute("gen_ai.usage.input_tokens", response.usage.prompt_tokens)
        span.set_attribute("gen_ai.usage.output_tokens", response.usage.completion_tokens)
        span.set_attribute("gen_ai.usage.total_tokens", response.usage.total_tokens)
        
        return response

# Test with a weather tool example
test_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "Temperature unit"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

result = make_llm_call_with_tools("What's the weather in Paris?", test_tools)
print("Tool definitions captured!")
print(f"   Captured {len(test_tools)} tool definition(s)")
if result.choices[0].message.tool_calls:
    print(f"   LLM called tool: {result.choices[0].message.tool_calls[0].function.name}")
else:
    print(f"   LLM Response: {result.choices[0].message.content}")

## 5. Part 3: Custom User-Defined Attributes

Add business context and custom metadata to traces using Fiddler's user-defined attribute patterns.

### Session-Level Attributes

These attributes should be added to **all spans** in a trace and use the pattern:
- `fiddler.session.user.{key}`

### Span-Level Attributes

These attributes are specific to individual spans:
- `fiddler.span.user.{key}`

In [ ]:
def travel_agent_with_custom_attributes(user_request: str, user_id: str, tier: str, region: str):
    """
    Enhanced travel agent with custom business attributes.
    """
    with tracer.start_as_current_span("travel_agent_chain") as root_span:
        # Required Fiddler attributes
        root_span.set_attribute("fiddler.span.type", "chain")
        root_span.set_attribute("gen_ai.agent.name", AGENT_NAME)
        root_span.set_attribute("gen_ai.agent.id", AGENT_ID)

        # ========== SESSION-LEVEL CUSTOM ATTRIBUTES ==========
        # These should be on ALL spans in the trace
        root_span.set_attribute("fiddler.session.user.user_id", user_id)
        root_span.set_attribute("fiddler.session.user.tier", tier)
        root_span.set_attribute("fiddler.session.user.region", region)
        # ===================================================

        # ========== SPAN-LEVEL CUSTOM ATTRIBUTES ==========
        # Business-specific metadata for this interaction
        root_span.set_attribute("fiddler.span.user.request_type", "travel_booking")
        root_span.set_attribute("fiddler.span.user.priority", "high" if tier == "premium" else "normal")
        # =================================================

        # Continue with normal agent logic...
        result = travel_agent(user_request)
        return result

# Test with custom attributes
print('\n' + '=' * 60)
print('Testing Custom Attributes')
print('=' * 60)
result = travel_agent_with_custom_attributes(
    user_request="Book a hotel in Tokyo for 2024-12-20",
    user_id="user_12345",
    tier="premium",
    region="APAC"
)
print('✅ Custom attributes added to trace')
print('   Session attributes: user_id, tier, region')
print('   Span attributes: request_type, priority')

## 6. Part 4: Conversation Management

Track multi-turn conversations using the `gen_ai.conversation.id` attribute.

In [ ]:
import uuid

def set_conversation_context(conversation_id: str):
    """
    Helper to add conversation ID to all spans in a conversation.
    In production, you would manage this through a context manager or middleware.
    """
    return conversation_id

def conversational_travel_agent(user_request: str, conversation_id: str = None):
    """
    Travel agent that tracks conversations.
    """
    # Generate or use provided conversation ID
    conv_id = conversation_id or str(uuid.uuid4())

    with tracer.start_as_current_span("travel_agent_chain") as root_span:
        # Required attributes
        root_span.set_attribute("fiddler.span.type", "chain")
        root_span.set_attribute("gen_ai.agent.name", AGENT_NAME)
        root_span.set_attribute("gen_ai.agent.id", AGENT_ID)

        # ========== CONVERSATION TRACKING ==========
        # Add conversation ID to track multi-turn interactions
        root_span.set_attribute("gen_ai.conversation.id", conv_id)
        # ==========================================

        result = travel_agent(user_request)
        result["conversation_id"] = conv_id
        return result

# Simulate multi-turn conversation
print('\n' + '=' * 60)
print('Multi-Turn Conversation Example')
print('=' * 60)

# First turn - generate new conversation ID
result1 = conversational_travel_agent("I need a hotel in San Francisco")
conv_id = result1["conversation_id"]
print(f'Turn 1: Conversation ID = {conv_id}')

# Second turn - use same conversation ID
result2 = conversational_travel_agent(
    "Also book me a flight from LA to San Francisco",
    conversation_id=conv_id
)
print(f'Turn 2: Conversation ID = {result2["conversation_id"]}')

print('\n✅ Multi-turn conversation tracked with consistent ID')

## 7. Part 5: Production Configuration

Configure OpenTelemetry for production deployments with:

- **Sampling**: Reduce volume by sampling traces
- **Batch Processing**: Optimize network usage
- **Compression**: Reduce data transmission size
- **Resource Attributes**: Add service metadata

In [ ]:
from opentelemetry.sdk.trace import sampling
from opentelemetry.sdk.resources import Resource
from opentelemetry.exporter.otlp.proto.http.trace_exporter import Compression

def create_production_tracer_provider():
    """
    Create a production-ready tracer provider with optimized settings.
    """
    # 1. Sampling: Sample 10% of traces
    sampler = sampling.TraceIdRatioBased(0.1)

    # 2. Resource attributes: Add service metadata
    resource = Resource.create({
        "service.name": "travel-agent-service",
        "service.version": "1.0.0",
        "deployment.environment": "production",
        "application.id": APPLICATION_UUID
    })

    # 3. Create provider with sampler and resource
    provider = TracerProvider(sampler=sampler, resource=resource)

    # 4. Configure OTLP exporter with compression
    otlp_exporter = OTLPSpanExporter(
        endpoint=otlp_endpoint,
        compression=Compression.Gzip  # Enable gzip compression
    )

    # 5. Batch processor with production settings
    batch_processor = BatchSpanProcessor(
        otlp_exporter,
        max_queue_size=500,           # Max spans in queue
        max_export_batch_size=50,     # Spans per export batch
        schedule_delay_millis=500,    # Export every 500ms
        export_timeout_millis=10000   # 10 second timeout
    )

    provider.add_span_processor(batch_processor)
    return provider

print('\n' + '=' * 60)
print('Production Configuration')
print('=' * 60)
print('✅ Production tracer provider configured')
print('   - Sampling: 10% of traces')
print('   - Compression: gzip enabled')
print('   - Batch size: 50 spans')
print('   - Export interval: 500ms')
print('   - Resource metadata: service.name, version, environment')
print('\n💡 Use this configuration in production deployments')

## 8. Part 6: Using FiddlerClient Alternative

If you have `fiddler-langgraph` installed, you can use `FiddlerClient.get_tracer()` for simplified setup.

In [ ]:
# Optional: Simplified setup with FiddlerClient
# Requires: pip install fiddler-langgraph

try:
    from fiddler_langgraph import FiddlerClient

    # Initialize FiddlerClient
    fdl_client = FiddlerClient(
        api_key=ACCESS_TOKEN,
        application_id=APPLICATION_UUID,
        url=FIDDLER_URL
    )

    # Get pre-configured tracer
    tracer_simplified = fdl_client.get_tracer()

    print('\n' + '=' * 60)
    print('FiddlerClient Alternative')
    print('=' * 60)
    print('✅ FiddlerClient setup complete')
    print('   This approach handles OTLP configuration automatically')
    print('   Use tracer_simplified for manual span creation')

except ImportError:
    print('\n' + '=' * 60)
    print('FiddlerClient Alternative (Optional)')
    print('=' * 60)
    print('⚠️  fiddler-langgraph not installed')
    print('   Install with: pip install fiddler-langgraph')
    print('   This provides simplified tracer setup')

## 9. Verify Traces in Fiddler

After running the examples above, traces should appear in your Fiddler application.

**Steps to verify:**

1. Navigate to your Fiddler instance
2. Go to **GenAI Apps** and select your application
3. Check that application status shows as **Active**
4. View traces in the **Traces** tab

**What to check:**

- ✅ Application shows as Active
- ✅ Traces appear within 1-2 minutes
- ✅ Span hierarchy: chain → LLM → tools
- ✅ Required attributes present:
  - `fiddler.span.type`
  - `gen_ai.agent.name`
  - `gen_ai.agent.id`
- ✅ LLM attributes:
  - `gen_ai.request.model`
  - `gen_ai.usage.*` (token counts)
- ✅ Tool attributes:
  - `gen_ai.tool.name`
  - `gen_ai.tool.input`
  - `gen_ai.tool.output`
- ✅ Custom attributes:
  - `fiddler.session.user.*`
  - `fiddler.span.user.*`
- ✅ Conversation tracking:
  - `gen_ai.conversation.id`

## 10. Troubleshooting Diagnostics

Run diagnostics to verify your configuration.

In [ ]:
import requests

print('\n' + '=' * 60)
print('DIAGNOSTIC CHECKS')
print('=' * 60)

# 1. Environment variables
print('\n1. Environment Variables:')
print(f'   OTEL_EXPORTER_OTLP_ENDPOINT: {os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT")}')
print(f'   OTEL_RESOURCE_ATTRIBUTES: {os.getenv("OTEL_RESOURCE_ATTRIBUTES")}')
print(f'   OTEL_EXPORTER_OTLP_HEADERS: {"Set" if os.getenv("OTEL_EXPORTER_OTLP_HEADERS") else "Not Set"}')

# 2. Network connectivity
print('\n2. Network Connectivity:')
try:
    response = requests.head(FIDDLER_URL, timeout=5)
    print(f'   ✅ Connection to {FIDDLER_URL}: Success (Status: {response.status_code})')
except Exception as e:
    print(f'   ❌ Connection to {FIDDLER_URL}: Failed ({str(e)})')

# 3. Configuration summary
print('\n3. Configuration Summary:')
print(f'   Fiddler URL: {FIDDLER_URL}')
print(f'   Application UUID: {APPLICATION_UUID}')
print(f'   Access Token: {"Set" if ACCESS_TOKEN else "Not Set"}')
print(f'   OpenAI API Key: {"Set" if OPENAI_API_KEY else "Not Set"}')

# 4. OpenTelemetry configuration
print('\n4. OpenTelemetry Status:')
print(f'   Tracer Provider: {"Initialized" if trace.get_tracer_provider() else "Not Initialized"}')
print(f'   OTLP Endpoint: {otlp_endpoint}')

print('\n' + '=' * 60)
print('\n💡 If traces are not appearing:')
print('   1. Verify credentials are correct')
print('   2. Check network connectivity')
print('   3. Ensure Application UUID is valid UUID4 format')
print('   4. Look for errors in console exporter output above')
print('   5. Wait 1-2 minutes for traces to appear in Fiddler')

## 11. Next Steps

Now that you have advanced OpenTelemetry integration working:

### Explore Fiddler Capabilities

- **Monitor Performance**: Track latency, token usage, and success rates
- **Custom Dashboards**: Create dashboards using your custom attributes
- **Alerts**: Set up alerts for errors, high latency, or anomalies
- **Cost Analysis**: Monitor LLM API costs through token tracking

### Consider SDKs for Specific Frameworks

- [Fiddler LangGraph SDK](https://docs.fiddler.ai/tutorials/llm-monitoring/langgraph-sdk-quick-start) - Auto-instrumentation for LangGraph/LangChain
- [Fiddler Strands SDK](https://docs.fiddler.ai/tutorials/llm-monitoring/strands-agent-quick-start) - Native Strands integration

### Production Deployment

- Implement the production configuration from Part 5
- Set up error handling and retry logic
- Configure sampling based on your volume
- Add environment-specific resource attributes
- Implement PII filtering if needed

### Resources

- [OpenTelemetry Quick Start Guide](https://docs.fiddler.ai/tutorials/llm-monitoring/opentelemetry-quick-start)
- [Getting Started: Agentic Monitoring](https://docs.fiddler.ai/getting-started/agentic-monitoring)
- [Fiddler Documentation](https://docs.fiddler.ai)
- [OpenTelemetry Python](https://opentelemetry.io/docs/instrumentation/python/)

---

**Need Help?**

- Email: support@fiddler.ai
- Documentation: https://docs.fiddler.ai